In [49]:
from Korpora import Korpora
import sys
sys.path.append('..')
from txt_cleaner.clean.master import MasterCleaner
from txt_cleaner.utils import *

In [2]:
Korpora.fetch('namuwikitext')

[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.train.zip
[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.train
[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.test.zip
[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.test
[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.dev.zip
[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.dev


In [296]:
corpus = Korpora.load('namuwikitext')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/namuwikitext
    References :

    나무위키의 덤프 데이터를 바탕을 제작한 wikitext 형식의 텍스트 파일입니다.
    학습 및 평가를 위하여 위키페이지 별로 train (99%), dev (0.5%), test (0.5%) 로 나뉘어져있습니다.


    # License
    CC BY-NC-SA 2.0 KR which Namuwiki dump dataset is licensed

[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.train.zip
[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.train
[Korpora] Corpus `namuwikitext` is already installed at /home/hscho/Korpora/namuwikitext/namuwikitext_20200302.test.zip
[Korpora] Corpus `namuwikitext` is already installed at /home/h

In [297]:
import mecab
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator

In [298]:
pos = mecab.MeCab()

def tokenize_pos(inp):
    if type(inp) == str:
        return pos.morphs(inp)
    if type(inp) == list:
        return [tokenize_pos(i) for i in inp]
# pos.morphs(['안녕하세요'])

In [299]:
tokenize_pos('안녕하세요'), tokenize_pos(['안녕하세요', '안녕?'])

(['안녕', '하', '세요'], [['안녕', '하', '세요'], ['안녕', '?']])

In [300]:
config = json_reader('../txt_cleaner/cleaner_config.json')


size 1 dictionary is read from ../txt_cleaner/cleaner_config.json


In [301]:
config

{'minimum_space_count': 5}

In [302]:
cleaner = MasterCleaner(config)
cleaner.cleaning('안녕하세요? 반갑습니다')

''

In [303]:
tokenize_pos(['안녕하세요', '안녕'])

[['안녕', '하', '세요'], ['안녕']]

In [327]:
ko_field = Field(tokenize = lambda e: tokenize_pos(e),
                init_token = '<sos>',
                eos_token = '<eos>',
                batch_first = True)
ko_field = Field(tokenize = lambda e: e.split(),
                init_token = '<sos>',
                eos_token = '<eos>',
                batch_first = True)

/home/hscho/anaconda3/envs/long36v/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [328]:
pos.morphs('안녕하세요')

['안녕', '하', '세요']

In [329]:
train = corpus.train.texts
ko_field.build_vocab([_ for _ in train if _])
# ko_field.build_vocab('안녕하세요')

In [330]:
tokenize_pos(list(train)[3])[:10]

['羽衣', '小', '町', '《', '아이돌', '마스터', '신데렐라', '걸즈', '》', '에']

In [334]:
preprocess = ko_field.preprocess(['안녕하세요', '안녕 반갑습니다'])
print(preprocess)
process = ko_field.process(preprocess)
process

['안녕하세요', '안녕 반갑습니다']


tensor([[   2,  117, 1079,   13,  142,  175,    3,    1,    1,    1],
        [   2,  117, 1079,    4,  149,  647,  274,   77,    7,    3]])